In [2]:
import torchvision.models as models
import torchvision
import torch 
import onnx
import onnxruntime as ort
import numpy as np
from openvino.inference_engine import IECore
import timm
import cv2

In [3]:
# input shape and batch size
INPUT_SHAPE = (3, 640, 640)
batch_size = 1

model_name = 'yolov5n'
onnx_model_path = f'{model_name}.onnx'

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [4]:
torch_model =  torch.hub.load('ultralytics/yolov5', 'yolov5n') #models.mobilenet_v3_large()
torch_model.eval()

Using cache found in C:\Users\14135/.cache\torch\hub\ultralytics_yolov5_master
C:\Users\14135\miniconda3\envs\torch_p37\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
YOLOv5  2022-2-23 torch 1.10.2 CPU

Fusing layers... 
Model Summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
           

In [5]:
def torch_to_onnx(model, path, batch_size, input_shape, simplify = True,
                dynamic_axes =  {'input' : {0 : 'batch_size'},    # variable length axes
                                 'output' : {0 : 'batch_size'}}):
    dummy_input = torch.randn(batch_size, *input_shape, requires_grad = True)

    torch.onnx.export(model,           # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  path,   # where to save the model (can be a file or file-like object)
                  training = False,
                  export_params = True,        # store the trained parameter weights inside the model file
                  opset_version = 11,          # the ONNX version to export the model to
                  do_constant_folding = True,  # whether to execute constant folding for optimization
                  input_names = ['images'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes = dynamic_axes)
    
    print(f"model saved to {path}")

    if simplify:
        import onnxsim

        model_onnx = onnx.load(path)
        model_onnx, check = onnxsim.simplify(
                    model_onnx,
                    dynamic_input_shape = True,
                    input_shapes = {'images': [batch_size, *input_shape]})
                    
        assert check, 'simplified check failed'
        onnx.save(model_onnx, onnx_model_path)
        print('simplifed model saved')


def load_onnx_model(path):
    model = onnx.load(path)
    onnx.checker.check_model(model)
    return model

In [6]:
torch_to_onnx(torch_model, onnx_model_path, batch_size, INPUT_SHAPE,
                                         dynamic_axes = {'images': {0: 'batch', 2: 'height', 3: 'width'},  # shape(1, 3, 640, 640)
                                                        'output': {0: 'batch', 1: 'anchors'}}  # shape(1, 25200, 85)
                                        )

onnx_model = load_onnx_model(onnx_model_path)

C:\Users\14135\miniconda3\envs\torch_p37\lib\site-packages\torch\onnx\utils.py:310: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn("It is recommended that constant folding be turned off ('do_constant_folding=False') "
C:\Users\14135/.cache\torch\hub\ultralytics_yolov5_master\models\yolo.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not gen

model saved to yolov5n.onnx
simplifed model saved


In [9]:
ort_session = ort.InferenceSession(onnx_model_path)

# define model input x to test onx vs torch
inputs = torch.randn(batch_size, *INPUT_SHAPE)

torch_out = torch_model(inputs)[0]

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(inputs)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0][0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

NameError: name 'torch_model' is not defined

In [29]:
!"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"

Python 3.7.11
[setupvars.bat] OpenVINO environment initialized


In [7]:
ie = IECore()
device = 'CPU'#'MYRIAD'
model = ie.read_network(model = onnx_model_path)
versions = ie.get_versions(device)
print(versions)

print(f"{device}")
print(f"MKLDNNPlugin version ......... {versions[device].major}.{versions[device].minor}")
print(f"Build ........... {versions[device].build_number}")

{'CPU': <class '__main__.Versions'>}
CPU
MKLDNNPlugin version ......... 2.1
Build ........... 2021.4.2-3976-0943ed67223-refs/pull/539/head


In [8]:
for input_key in model.input_info:
     print(model.input_info[input_key].input_data.shape)

for input_key in model.input_info:
    input_name = input_key
    model.input_info[input_key].precision = 'FP16' if device == 'MYRIAD' else 'FP32'
    break
assert (len(model.input_info.keys()) == 1 or len(model.input_info.keys()) == 2), "Sample supports topologies only with 1 or 2 inputs"


[]


In [10]:
_input_shape = str((1, *INPUT_SHAPE)).replace(' ', '')
# jupyter bash can be pretty buggy, make sure to copy and paste input shape and model path in 
print(_input_shape)
print(onnx_model_path) ####MAKE SURE TO COPY THIS INTO THE MODEL PATH 
# only run when model is updated 
# get conv output layers with https://netron.app webste

!python "C:\\Program Files (x86)\\Intel\\openvino_2021\\deployment_tools\\model_optimizer\\mo.py" \
    --input_model "C:\\Users\\14135\\Desktop\\birdnet_torch\\yolov5n.onnx" \
    --output_dir "C:\\Users\\14135\\Desktop\\birdnet_torch\\models" \
    --input_shape (1,3,640,640) \
    --output Conv_199,Conv_510,Conv_821 \
    --data_type FP16 \
    --scale 255

(1,3,640,640)
yolov5n.onnx
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	C:\\Users\\14135\\Desktop\\birdnet_torch\\yolov5n.onnx
	- Path for generated IR: 	C:\\Users\\14135\\Desktop\\birdnet_torch\\models
	- IR output name: 	yolov5n
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Conv_199,Conv_510,Conv_821
	- Input shapes: 	(1,3,640,640)
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	255.0
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
	- Inference Engine found in: 	C:\Users\14135\miniconda3\envs\torch_p37\lib\site-packages\openvino
Inference Engine version: 	2021.4.2-3976-0943ed67223-refs/pull/539/head
Model Optimizer version: 	2021.4.2-3974-e2a469a3450-releases/2021/4
[

[ WARNING ]  Const node 'Resize_119/Add_input_port_1/value267210789' returns shape values of 'float64' type but it must be integer or float32. During Elementwise type inference will attempt to cast to float32
[ WARNING ]  Const node 'Resize_141/Add_input_port_1/value270610792' returns shape values of 'float64' type but it must be integer or float32. During Elementwise type inference will attempt to cast to float32
[ WARNING ]  Changing Const node 'Resize_119/Add_input_port_1/value267210960' data type from float16 to <class 'numpy.float32'> for Elementwise operation
[ WARNING ]  Changing Const node 'Resize_141/Add_input_port_1/value270611062' data type from float16 to <class 'numpy.float32'> for Elementwise operation


In [11]:
net = ie.read_network(model = f'models\\{model_name}.xml', weights = f'models\\{model_name}.bin')

exec_net = ie.load_network(network = net, device_name = device, num_requests=2)

In [222]:
import cv2
import numpy as np

img = cv2.imread("bus.jpg")
print(img.shape)
img = img[..., ::-1]
img = cv2.resize(img, (640, 640))

cv2.imshow('window', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(1080, 810, 3)


In [221]:
img = torchvision.io.read_image("bus.jpg")
img = torchvision.transforms.functional.resize(img, [640, 640])
print(img.shape)
img = np.expand_dims(to_numpy(img), axis = 0)
inputs = {'images': img}

torch.Size([3, 640, 640])


In [129]:
outputs = exec_net.infer(inputs=inputs)
results = []
for layer_name, out in outputs.items():
    results.append(out)

In [130]:
import numpy as np

YOLOV5N_ANCHORS = [
    [10,13, 16,30, 33,23],  # P3/8
    [30,61, 62,45, 59,119],  # P4/16
    [116,90, 156,198, 373,326],  # P5/32 
    ]

def create_detector(num_classes = 80,
    anchors = YOLOV5N_ANCHORS,
    ):
    num_outputs = num_classes + 5 # num outpus per anchor
    num_layers = len(anchors) # number of prediction layers 
    num_anchors = len(anchors[0]) //2
    anchors = anchors
    grid = [np.zeros(1)] * num_layers  # init grid
    anchor_grid = [np.zeros(1)] * num_layers # init anchor grid

    def detect(predictions): # call on one image
        z = []
        for i, pred in enumerate(predictions): # for each prediction layer
            batch_size, _, ny, nx = pred.shape

            x = np.reshape(pred, (batch_size, num_anchors, num_outputs, ny, nx))
            x = np.transpose(x, (0, 1, 3, 4, 2))

            y = 1 / (1 + np.exp(-x))

            y[..., 0:2] = (y[..., 0:2] * 2 - 0.5 + grid[i]) # * stride[i]  # xy
            y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * anchor_grid[i]  # wh

            y = np.reshape(y, (batch_size, -1, num_outputs))
            z.append(y)
            
        return np.concatenate(z, axis = 1)
        
    return detect

In [220]:
detector = create_detector()
print(results[0].shape)
x = detector(results)
np.max(x[..., 4])

(1, 255, 80, 80)
(1, 3, 80, 80, 85)
(1, 3, 40, 40, 85)
(1, 3, 20, 20, 85)


0.18959783

In [218]:
x = detector(results)
y = non_max_suppression(x, conf_thres= .1)

(1, 3, 80, 80, 85)
(1, 3, 40, 40, 85)
(1, 3, 20, 20, 85)
(4, 1) (4, 6)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [8]:

%timeit exec_net.infer(inputs=inputs)
# yolov5s time: 262 ms
# yolov5n time: 138 ms
# yolov5s torch laptop cpu time: 442 ms 
# yolov5n torch laptop cpu time: 204 ms
# yolov5n onnx laptop cpu time: 55 ms
# yolov5n openvino laptop cpu: 38.6 ms

38.6 ms ± 2.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [77]:
x = torch.from_numpy(img)

%timeit torch_model(x)

237 ms ± 9.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
ort_session = ort.InferenceSession(onnx_model_path)
ort_inputs = {ort_session.get_inputs()[0].name: img}
ort_outs = ort_session.run(None, ort_inputs)

In [30]:
ort_outs[0].shape

(1, 25200, 85)

In [29]:
(255*80*80 + 255*40*40 + 255*20*20) //85

25200

In [31]:
85*3

255

In [12]:
%timeit ort_session.run(None, ort_inputs)

NameError: name 'ort_inputs' is not defined

In [58]:
!python "C:\\Program Files (x86)\\Intel\\openvino_2021\\deployment_tools\\model_optimizer\\mo.py" --help

usage: main.py [options]

optional arguments:
  -h, --help            show this help message and exit
  --framework {tf,caffe,mxnet,kaldi,onnx}
                        Name of the framework used to train the input model.

Framework-agnostic parameters:
  --input_model INPUT_MODEL, -w INPUT_MODEL, -m INPUT_MODEL
                        Tensorflow*: a file with a pre-trained model (binary
                        or text .pb file after freezing). Caffe*: a model
                        proto file with model weights
  --model_name MODEL_NAME, -n MODEL_NAME
                        Model_name parameter passed to the final create_ir
                        transform. This parameter is used to name a network in
                        a generated IR and output .xml/.bin files.
  --output_dir OUTPUT_DIR, -o OUTPUT_DIR
                        Directory that stores the generated IR. By default, it
                        is the directory from where the Model Optimizer is
                        l